<a href="https://colab.research.google.com/github/GuilhermeMoraesLS/Projeto_SE_Maxmalize/blob/main/prototipo_policeargos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ============================================================
# 🚓 POLICE ARGOS v2.3 — OCR Only com detecção de ROI
# ============================================================

!pip install -q easyocr opencv-python-headless

import cv2
import easyocr
import numpy as np
import os
from google.colab import files as colab_files
import re

# ============================================================
# CONFIGURAÇÕES
# ============================================================

placas_irregulares = ["ABC1D23", "R102A18", "XYZ9Z99", "PLR3D97", "LAD8945"]
non_plate_words = ["BRASIL","MERCOSUL","SP","SÃO","PAULO","RIO","JANEIRO",
                   "RJ","MG","PR","SC","RS","BA","PE","CE","DF"]

MIN_PLATE_LENGTH = 6
CONFIDENCE_OCR = 0.2
reader = easyocr.Reader(['pt','en'], gpu=True)

# ============================================================
# FUNÇÕES AUXILIARES
# ============================================================

def validar_formato_placa(texto):
    texto = texto.upper().replace(" ","").replace("-","")
    padrao_antigo = re.match(r'^[A-Z]{3}\d{4}$', texto)
    padrao_mercosul = re.match(r'^[A-Z]{3}\d[A-Z]\d{2}$', texto)
    return bool(padrao_antigo or padrao_mercosul), texto

def limpar_texto_ocr(texto):
    return ''.join(filter(str.isalnum,texto)).upper()

def corrigir_ocr(texto):
    mapping = {'O':'0','I':'1','L':'1','Z':'2','S':'5'}
    return ''.join(mapping.get(c,c) for c in texto)

def verificar_placa_irregular(texto):
    return texto in placas_irregulares

def encontrar_roi_placa(img):
    """Detecta possíveis regiões de placas usando contornos"""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    # Realçar bordas
    edges = cv2.Canny(blur,100,200)
    # Dilatação para unir partes da placa
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    dilated = cv2.dilate(edges,kernel,iterations=1)

    contours,_ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    rois = []
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        aspect_ratio = w/h
        if 2 < aspect_ratio < 6 and w>60 and h>15:  # proporção típica de placa
            roi = img[y:y+h,x:x+w]
            rois.append((roi,(x,y,w,h)))
    return rois

# ============================================================
# UPLOAD DA IMAGEM
# ============================================================

uploaded = colab_files.upload()
if not uploaded:
    print("❌ Nenhum arquivo enviado.")
else:
    image_path = list(uploaded.keys())[0]
    image = cv2.imread(image_path)

    if image is None:
        print("❌ Erro ao carregar imagem.")
    else:
        rois = encontrar_roi_placa(image)
        melhor_texto = ""
        melhor_conf = 0
        roi_final = None

        # ============================================================
        # OCR nas regiões detectadas
        # ============================================================
        for roi,(x,y,w,h) in rois:
            gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            # OCR
            ocr_result = reader.readtext(thresh, detail=1)
            for (bbox,text,prob) in ocr_result:
                cleaned = limpar_texto_ocr(text)
                cleaned = corrigir_ocr(cleaned)
                if cleaned in non_plate_words: continue
                if len(cleaned)<MIN_PLATE_LENGTH or prob<CONFIDENCE_OCR: continue
                eh_valida, placa_formatada = validar_formato_placa(cleaned)
                if eh_valida and prob>melhor_conf:
                    melhor_texto = placa_formatada
                    melhor_conf = prob
                    roi_final = (x,y,w,h)

        # ============================================================
        # RESULTADO FINAL
        # ============================================================
        if melhor_texto:
            status = "IRREGULAR" if verificar_placa_irregular(melhor_texto) else "REGULAR"
            cor_status = (0,0,255) if status=="IRREGULAR" else (0,255,0)
            if roi_final:
                x,y,w,h = roi_final
                cv2.rectangle(image,(x,y),(x+w,y+h),cor_status,3)
            cv2.putText(image,f"{status}: {melhor_texto}",(50,50),
                        cv2.FONT_HERSHEY_SIMPLEX,1.2,cor_status,3)
            print(f"✅ Placa detectada: {melhor_texto} | Status: {status} | Confiança: {melhor_conf:.2%}")
        else:
            cv2.putText(image,"NENHUMA PLACA DETECTADA",(50,50),
                        cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            print("❌ Nenhuma placa identificada")

        # ============================================================
        # SALVAR E BAIXAR RESULTADO
        # ============================================================
        output_path = os.path.splitext(image_path)[0]+"_ocr_roi.jpg"
        cv2.imwrite(output_path,image)
        colab_files.download(output_path)
        print(f"📸 Imagem processada salva: {output_path}")


Saving 2019-10-30-placa-mercosul-nova_police_argos.jpg to 2019-10-30-placa-mercosul-nova_police_argos (3).jpg
✅ Placa detectada: BRA2E19 | Status: REGULAR | Confiança: 99.11%


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📸 Imagem processada salva: 2019-10-30-placa-mercosul-nova_police_argos (3)_ocr_roi.jpg
